In [1]:
import os
import glob
import numpy as np
import xarray as xr
import rioxarray as riox
from datetime import datetime

In [2]:
print('   setting up Local cluster...')
from dask.distributed import LocalCluster, Client
import dask
cluster = LocalCluster(n_workers=1, threads_per_worker=2)
client = Client(cluster)

   setting up Local cluster...


2022-06-21 09:41:19,157 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-2dx4ntz8', purging
2022-06-21 09:41:19,163 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-74vj23ir', purging
2022-06-21 09:41:19,167 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-8cqu0ncr', purging
2022-06-21 09:41:19,172 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-b81dav7r', purging
2022-06-21 09:41:19,177 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-dl05ji2h', purging
2022-06-21 09:41:19,182 - distributed.diskutils - INFO - Found stale lock file and directory '/

In [3]:
prefix = 'cper'
tifDIR = os.path.join('data/hls_nrt', prefix)

In [4]:
yr = 2022
da_bare = riox.open_rasterio(os.path.join(tifDIR,
                                         'hls_cover',
                                          prefix + '_hls_BARE_' + 
                                          str(yr) + '.nc')).rename({'time': 'date'})#.chunk(sngl_chunks)
da_green = riox.open_rasterio(os.path.join(tifDIR,
                                         'hls_cover',
                                          prefix + '_hls_GREEN_' + 
                                          str(yr) + '.nc')).rename({'time': 'date'})#.chunk(sngl_chunks)
da_sd = riox.open_rasterio(os.path.join(tifDIR,
                                         'hls_cover',
                                          prefix + '_hls_SD_' + 
                                          str(yr) + '.nc')).rename({'time': 'date'})#.chunk(sngl_chunks)
da_litt = riox.open_rasterio(os.path.join(tifDIR,
                                         'hls_cover',
                                          prefix + '_hls_LITT_' + 
                                          str(yr) + '.nc')).rename({'time': 'date'})#.chunk(sngl_chunks)
da_bm = riox.open_rasterio(os.path.join(tifDIR,
                                        'hls_biomass',
                                        prefix + '_hls_bm_' +
                                        str(yr) + '.nc')).rename({'time': 'date'})#.chunk(sngl_chunks)
da_bm_se = riox.open_rasterio(os.path.join(tifDIR,
                                        'hls_biomass',
                                        prefix + '_hls_bm_se_' +
                                        str(yr) + '.nc')).rename({'time': 'date'})#.chunk(sngl_chunks)
da_ndvi = riox.open_rasterio(os.path.join(tifDIR,
                                        'hls_ndvi',
                                        prefix + '_hls_ndvi_' +
                                        str(yr) + '.nc')).rename({'time': 'date'})#.chunk(sngl_chunks)
da_bare.name = 'BARE'
da_green.name = 'GREEN'
da_sd.name = 'SD'
da_litt.name = 'LITT'
da_bm.name = 'Biomass'
da_bm_se.name = 'Biomass_SE'
da_ndvi.name = 'NDVI'

In [5]:
da_bare = da_bare.where(da_bare > 0.0, other=0.0)
da_bare = da_bare.where(da_bare < 1.0, other=1.0)

da_green = da_green.where(da_green > 0.0, other=0.0)
da_green = da_green.where(da_green < 1.0, other=1.0)

da_sd = da_sd.where(da_sd > 0.0, other=0.0)
da_sd = da_sd.where(da_sd < 1.0, other=1.0)

da_litt = da_litt.where(da_litt > 0.0, other=0.0)
da_litt = da_litt.where(da_litt < 1.0, other=1.0)

cov_mask = da_bare.notnull() & da_green.notnull() & da_sd.notnull() & da_litt.notnull()

In [6]:
ds = xr.merge([da_bare.where(cov_mask), 
               da_green.where(cov_mask), 
               da_sd.where(cov_mask),
               da_litt.where(cov_mask),
               da_bm.where((da_bm >= 0) & (da_bm < 5000)),
               da_bm_se.where((da_bm >= 0) & (da_bm < 5000)),
               da_ndvi.where((da_ndvi >= 0) & (da_ndvi <= 1))])
ds = ds.where(ds != -9999.)

In [7]:
#ds['date'] = [datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S') for x in ds['date'].values]

In [8]:
#ds['date'] = ds['date'].dt.date
ds['DRY'] = ds['SD'] + ds['LITT']

In [9]:
ds.to_netcdf('data/gcloud/hls_cper_2022_gcloud.nc')